In [ ]:
import ccxt
import pandas as pd
import os

# --- Configurable Parameters ---
TRADE_AMOUNT = 10  # Trade amount in USD
DEFAULT_FEE = 0.001  # Default maker/taker fee (0.1% per trade)
DEFAULT_LIQUIDITY = 0  # Default liquidity if missing
RESULTS_FILE = "arbitrage_results.csv"

# --- Function to Fetch Market Data (Including Liquidity) ---
def get_market_data(exchange):
    """Fetches all USDT trading pairs and their quoteVolume (liquidity) from an exchange."""
    try:
        markets = exchange.load_markets()
        market_data = {}

        for s, data in markets.items():
            if "USDT" in s:
                # Try multiple locations for liquidity data, default to 0 if missing
                quote_volume = (
                    float(data.get("info", {}).get("quoteVolume", 0)) or
                    float(data.get("quoteVolume", 0)) or
                    float(data.get("baseVolume", 0)) or
                    float(data.get("info", {}).get("volume", 0)) or
                    DEFAULT_LIQUIDITY
                )
                market_data[s] = quote_volume

        return market_data  # Dictionary {symbol: liquidity}
    
    except Exception as e:
        print(f"⚠️ Error fetching market data for {exchange.name}: {e}")
        return {}

# --- Function to Get Exchange Info ---
def get_exchange_info(exchange_name):
    """Fetches trading information for a given exchange, including fees and liquidity data."""
    try:
        exchange = getattr(ccxt, exchange_name)()

        # Always use default fee values (0.1% per trade)
        trading_fees = {"maker_fee": DEFAULT_FEE, "taker_fee": DEFAULT_FEE}

        # Fetch market liquidity data
        market_data = get_market_data(exchange)

        return {
            "Exchange": exchange_name,
            "Trading Fees": trading_fees,
            "Market Data": market_data  # Dictionary {symbol: liquidity}
        }
    
    except Exception as e:
        print(f"⚠️ Could not retrieve data for {exchange_name}: {e}")
        return {"Error": str(e)}

# --- Function to Compare Fees, Spreads, and Liquidity ---
def compare_fees_spreads_liquidity(exchange1_name, exchange2_name):
    """Compares trading fees, spreads, and liquidity for common USDT pairs between two exchanges."""
    if exchange1_name == exchange2_name:
        raise ValueError("❌ Both exchanges are the same. Please enter two different exchanges.")

    if not hasattr(ccxt, exchange1_name) or not hasattr(ccxt, exchange2_name):
        raise ValueError("❌ One or both exchanges are not listed in CCXT. Check your inputs.")

    # Fetch exchange information
    exchange1_info = get_exchange_info(exchange1_name)
    exchange2_info = get_exchange_info(exchange2_name)

    # Find common trading symbols
    symbols1 = set(exchange1_info.get("Market Data", {}).keys())
    symbols2 = set(exchange2_info.get("Market Data", {}).keys())
    common_symbols = list(symbols1 & symbols2)

    print(f"✅ Found {len(common_symbols)} common USDT trading pairs between {exchange1_name} and {exchange2_name}")

    if not common_symbols:
        raise ValueError(f"❌ No common **USDT** trading pairs found between {exchange1_name} and {exchange2_name}.")

    # Connect to exchanges
    exchange1 = getattr(ccxt, exchange1_name)()
    exchange2 = getattr(ccxt, exchange2_name)()

    results = []

    for symbol in common_symbols:
        print(f"🔄 Fetching data for {symbol}...")

        try:
            # Fetch latest prices
            price1 = exchange1.fetch_ticker(symbol).get("last", None)
            price2 = exchange2.fetch_ticker(symbol).get("last", None)

            # Ensure prices are valid
            if price1 is None or price2 is None:
                print(f"⚠️ Missing price data for {symbol}, skipping...")
                continue

            # Calculate spread percentage
            spread = abs(price1 - price2) / ((price1 + price2) / 2) * 100

            # Get trading fees (Default to 0.1% per trade)
            fee1 = exchange1_info["Trading Fees"]
            fee2 = exchange2_info["Trading Fees"]

            # Get liquidity (24h volume), use default if missing
            liquidity1 = exchange1_info["Market Data"].get(symbol, DEFAULT_LIQUIDITY)
            liquidity2 = exchange2_info["Market Data"].get(symbol, DEFAULT_LIQUIDITY)

            # Calculate potential profit
            potential_profit = calculate_potential_profit(price1, price2, fee1["taker_fee"], fee2["taker_fee"], TRADE_AMOUNT)

            results.append({
                "Symbol": symbol,
                "Exchange 1": exchange1_name,
                "Exchange 2": exchange2_name,
                "Price 1": price1,
                "Price 2": price2,
                "Average Price": (price1 + price2) / 2,
                "Spread (%)": spread,
                "Exchange 1 Maker Fee": fee1["maker_fee"],
                "Exchange 1 Taker Fee": fee1["taker_fee"],
                "Exchange 2 Maker Fee": fee2["maker_fee"],
                "Exchange 2 Taker Fee": fee2["taker_fee"],
                "Exchange 1 Liquidity (24h Vol)": liquidity1,
                "Exchange 2 Liquidity (24h Vol)": liquidity2,
                "Potential Profit ($)": potential_profit
            })

        except Exception as e:
            print(f"⚠️ Error fetching data for {symbol}: {e}")
            continue  # Skip symbols that cause errors

    # Convert to a DataFrame for better visualization
    df = pd.DataFrame(results)

    # Replace NaN values with "N/A"
    df.fillna("N/A", inplace=True)

    if df.empty:
        raise ValueError("❌ No valid trading pairs with price and liquidity data found.")

    return df

# --- Function to Calculate Potential Profit ---
def calculate_potential_profit(price1, price2, fee1, fee2, trade_amount):
    """Calculates the potential profit for a given trade amount."""
    
    # Determine Buy & Sell Prices
    buy_price = min(price1, price2)
    sell_price = max(price1, price2)

    # Determine Fees
    buy_fee = fee1 if buy_price == price1 else fee2
    sell_fee = fee2 if sell_price == price2 else fee1

    # Calculate BTC Amount Bought
    btc_amount = trade_amount / buy_price

    # Calculate Trading Fees
    buy_fee_cost = buy_price * btc_amount * buy_fee
    sell_fee_cost = sell_price * btc_amount * sell_fee
    total_fees = buy_fee_cost + sell_fee_cost

    # Calculate Raw Profit
    raw_profit = (sell_price - buy_price) * btc_amount

    # Calculate Final Profit (After Fees)
    final_profit = raw_profit - total_fees
    return round(final_profit, 4)  # Round to 4 decimal places for clarity

# --- Run & Save Results ---
if not os.path.exists(RESULTS_FILE):
    try:
        exchange1 = "binance"
        exchange2 = "bybit"
        result_df = compare_fees_spreads_liquidity(exchange1, exchange2)

        # Save results
        result_df.to_csv(RESULTS_FILE, index=False)
        print(f"📁 Results saved to {RESULTS_FILE}")

        # Display the results
        display(result_df)

    except ValueError as e:
        print(e)
else:
    result_df = pd.read_csv(RESULTS_FILE)
    print("✅ Loaded saved results.")


✅ Found 565 common USDT trading pairs between binance and bybit
⚠️ Error fetching data for HNT/USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for XEM/USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for REN/USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for BTT/USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for SC/USDT:USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for BEAM/USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for DAI/USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for CVC/USDT:USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for OMG/USDT: unsupported operand type(s) for -: 'NoneType' and 'float'
⚠️ Error fetching data for WAVES/USDT: unsupported operand type(s) for 

,Symbol,Exchange 1,Exchange 2,Price 1,Price 2,Average Price,Spread (%),Exchange 1 Maker Fee,Exchange 1 Taker Fee,Exchange 2 Maker Fee,Exchange 2 Taker Fee,Exchange 1 Liquidity (24h Vol),Exchange 2 Liquidity (24h Vol),Potential Profit ($)
0,TNSR/USDT,binance,bybit,0.429500,0.42950,0.429500,0.000000,0.001,0.001,0.001,0.001,0.0,0.0,-0.0200
1,ARK/USDT:USDT,binance,bybit,0.366000,0.36590,0.365950,0.027326,0.001,0.001,0.001,0.001,0.0,0.0,-0.0173
2,CAKE/USDT,binance,bybit,1.734000,1.73200,1.733000,0.115407,0.001,0.001,0.001,0.001,0.0,0.0,-0.0085
3,CGPT/USDT:USDT,binance,bybit,0.108940,0.10892,0.108930,0.018360,0.001,0.001,0.001,0.001,0.0,0.0,-0.0182
4,BOME/USDT,binance,bybit,0.001589,0.00159,0.001589,0.062913,0.001,0.001,0.001,0.001,0.0,0.0,-0.0137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,ADA/USDT:USDT,binance,bybit,0.814700,0.81490,0.814800,0.024546,0.001,0.001,0.001,0.001,0.0,0.0,-0.0175
551,SNT/USDT:USDT,binance,bybit,0.036660,0.02656,0.031610,31.951914,0.001,0.001,0.001,0.001,0.0,0.0,3.7789
552,RED/USDT:USDT,binance,bybit,0.640700,0.64060,0.640650,0.015609,0.001,0.001,0.001,0.001,0.0,0.0,-0.0184
553,FET/USDT,binance,bybit,0.589000,0.58800,0.588500,0.169924,0.001,0.001,0.001,0.001,0.0,0.0,-0.0030


In [5]:
top_10_results = result_df.sort_values(by="Potential Profit ($)", ascending=False).head(10)
display(top_10_results)

print(top_10_results['Symbol'].values)

,Symbol,Exchange 1,Exchange 2,Price 1,Price 2,Average Price,Spread (%),Exchange 1 Maker Fee,Exchange 1 Taker Fee,Exchange 2 Maker Fee,Exchange 2 Taker Fee,Exchange 1 Liquidity (24h Vol),Exchange 2 Liquidity (24h Vol),Potential Profit ($)
474,NEIRO/USDT,binance,bybit,0.000290,0.039120,0.019705,197.057894,0.001,0.001,0.001,0.001,0.0,0.0,1338.2109
40,TST/USDT,binance,bybit,0.069400,0.003380,0.036390,181.423468,0.001,0.001,0.001,0.001,0.0,0.0,195.1101
501,MAVIA/USDT:USDT,binance,bybit,1.492900,0.139400,0.816150,165.839613,0.001,0.001,0.001,0.001,0.0,0.0,96.9776
108,CTK/USDT:USDT,binance,bybit,1.010600,0.388400,0.699500,88.949249,0.001,0.001,0.001,0.001,0.0,0.0,15.9835
366,GLMR/USDT:USDT,binance,bybit,0.259200,0.100100,0.179650,88.561091,0.001,0.001,0.001,0.001,0.0,0.0,15.8582
112,MDT/USDT:USDT,binance,bybit,0.063310,0.027860,0.045585,77.766809,0.001,0.001,0.001,0.001,0.0,0.0,12.6916
44,RAD/USDT:USDT,binance,bybit,1.687400,0.747500,1.217450,77.202349,0.001,0.001,0.001,0.001,0.0,0.0,12.5413
58,DGB/USDT:USDT,binance,bybit,0.019220,0.009037,0.014129,72.074176,0.001,0.001,0.001,0.001,0.0,0.0,11.2369
454,IDEX/USDT:USDT,binance,bybit,0.056100,0.030460,0.043280,59.242144,0.001,0.001,0.001,0.001,0.0,0.0,8.3892
394,SLP/USDT:USDT,binance,bybit,0.003515,0.001910,0.002713,59.170507,0.001,0.001,0.001,0.001,0.0,0.0,8.3747


['NEIRO/USDT' 'TST/USDT' 'MAVIA/USDT:USDT' 'CTK/USDT:USDT'
 'GLMR/USDT:USDT' 'MDT/USDT:USDT' 'RAD/USDT:USDT' 'DGB/USDT:USDT'
 'IDEX/USDT:USDT' 'SLP/USDT:USDT']
